In [1]:
!pip install -q transformers datasets accelerate peft huggingface_hub

import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

In [2]:
#ds = load_dataset("rsilveira79/soprano_dpo_pairs", split="train")
#ds = ds.rename_columns({"question": "instruction", "chosen": "output"})
ds = load_dataset("json", data_files="/content/tony_soprano_finetune.jsonl", split="train")


def format_sample(example):
    return {
        "text": f"Instruction: {example['instruction']}\nResponse: {example['response']}"
    }

ds = ds.map(format_sample)


#model_name = "microsoft/phi-2"
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    tok = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tok["labels"] = tok["input_ids"]
    return tok

tokenized_ds = ds.map(tokenize, batched=True)
tokenized_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.config.use_cache = False

trainable params: 12,582,912 || all params: 3,833,662,464 || trainable%: 0.3282


In [8]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    #target_modules=["q_proj", "v_proj"],
    target_modules=["qkv_proj", "o_proj", "gate_up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 12,582,912 || all params: 3,833,662,464 || trainable%: 0.3282


In [9]:
training_args = TrainingArguments(
    output_dir="./soprano_mistral_lora_fp16",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    save_total_limit=2,
    report_to="none"
)

In [10]:
os.environ["WANDB_DISABLED"] = "true"
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [11]:
trainer.train()


Step,Training Loss
10,7.013300
20,1.353900
30,1.033100
40,0.987900
50,0.995300
60,0.924300
70,0.781100
80,1.042400
90,0.887100
100,0.851900


Step,Training Loss
10,7.013300
20,1.353900
30,1.033100
40,0.987900
50,0.995300
60,0.924300
70,0.781100
80,1.042400
90,0.887100
100,0.851900


TrainOutput(global_step=522, training_loss=0.9057971951605259, metrics={'train_runtime': 1357.6292, 'train_samples_per_second': 3.06, 'train_steps_per_second': 0.384, 'total_flos': 2.383810177794048e+16, 'train_loss': 0.9057971951605259, 'epoch': 3.0})

In [ ]:
# save LoRA weights in new dir within colab
model.save_pretrained("./loraAdapter")